# Testing Notebook 02:

In [74]:
from typing import List
import pandas as pd 
import numpy as np
import plotly.express as px

from statdepth.depth.depth import FunctionalDepth

In [75]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [76]:
df = pd.DataFrame(np.random.rand(5, 10), columns=list('ZXCVBNMLKJ'))
px.line(df)


In [84]:
col = [2.6,3,4,4,4,4.3,4.2,3,4,3,2.5,3,3,3,3.5, 4] 
col = [k * 3 for k in col]
df1 = pd.DataFrame()
for i in range(30):
    l = np.random.rand()
    df1[f'{i}'] = [k*l for k in col]
px.line(df1)

In [91]:
df = pd.DataFrame(np.random.rand(30, 30))

bd = FunctionalDepth([df1], K=2, J=3, relax=False)
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.921837,0.957439,0.173060,0.932006,0.993304,0.891399,0.485117,0.348399,0.665993,0.916444,...,0.933231,0.462914,0.623793,0.456886,0.812075,0.809315,0.246162,0.132592,0.878138,0.871714
1,0.780021,0.801431,0.470112,0.705003,0.677193,0.559722,0.105311,0.592770,0.405123,0.467554,...,0.384458,0.182040,0.946828,0.538162,0.397935,0.554984,0.556613,0.805442,0.130843,0.117239
2,0.894631,0.899529,0.125659,0.759784,0.265999,0.433187,0.458749,0.045776,0.584112,0.826297,...,0.838311,0.934827,0.701124,0.074685,0.070102,0.136491,0.102261,0.821641,0.117899,0.569885
3,0.700979,0.378872,0.913729,0.356408,0.649634,0.743874,0.626572,0.085651,0.515258,0.130660,...,0.818700,0.777556,0.775813,0.467111,0.992660,0.441006,0.328714,0.262263,0.163338,0.903547
4,0.196241,0.446298,0.674436,0.622035,0.454015,0.514978,0.877982,0.711527,0.238492,0.100524,...,0.466447,0.125563,0.087927,0.646892,0.291489,0.185685,0.844585,0.809188,0.051584,0.723765
5,0.060233,0.724473,0.776515,0.819426,0.487692,0.764445,0.540632,0.575703,0.062903,0.189285,...,0.463674,0.284204,0.804950,0.165164,0.480099,0.338141,0.033514,0.402905,0.708595,0.233457
6,0.747655,0.934822,0.941072,0.610715,0.378881,0.357328,0.103664,0.501350,0.202279,0.588611,...,0.872592,0.538412,0.371522,0.842158,0.447428,0.958828,0.133338,0.646003,0.171338,0.264859
7,0.524678,0.519483,0.419194,0.267944,0.789264,0.659906,0.429446,0.572635,0.227369,0.787413,...,0.898367,0.040945,0.017775,0.720670,0.807227,0.802180,0.136348,0.015948,0.787564,0.279639
8,0.922921,0.758057,0.046197,0.031837,0.093666,0.857008,0.168757,0.254789,0.231505,0.284044,...,0.706182,0.131539,0.563478,0.186703,0.947577,0.129277,0.112619,0.306063,0.612581,0.365455
9,0.261280,0.040866,0.203399,0.627931,0.583763,0.329964,0.797796,0.179156,0.863304,0.220024,...,0.029054,0.934663,0.076794,0.953701,0.249314,0.370275,0.626587,0.434193,0.670243,0.322181


In [92]:
bd.sorted()

25    1.103388
22    1.103388
27    1.083448
4     1.083448
15    1.069322
29    1.043567
3     1.043567
19    1.009501
14    0.983745
2     0.952724
13    0.949679
21    0.913118
10    0.813416
18    0.810920
26    0.804281
6     0.770215
20    0.733654
23    0.646131
11    0.633951
28    0.631456
7     0.531754
16    0.531754
9     0.412111
12    0.406845
5     0.287202
17    0.259226
24    0.139583
0     0.139583
8     0.000000
1     0.000000
dtype: float64

In [93]:
bd.deepest(n=2)

25    1.103388
22    1.103388
dtype: float64

In [96]:
bd.plot_deepest(n=1)

In [27]:
import plotly.graph_objects as go

def _plot(df, deep_or_outlying: pd.Series) -> None:
    cols = df.columns
    x = df.index

    # We use deep_or_outlying.index to get the columns because 
    # deep_or_outlying is a Series indexed by the original columns
    
    print(deep_or_outlying.index)
    data=[go.Scatter(x=x, y=df.loc[:, y], mode='lines+markers', marker_color='Blue') for y in cols]
    data.extend([go.Scatter(x=x, y=df.loc[:, y], mode='lines+markers', marker_color='Red') for y in deep_or_outlying.index])

    fig = go.Figure(data=data)
    fig.update_layout(showlegend=False)

    fig.show()


In [32]:
_plot(df=df, deep_or_outlying=bd.deepest(n=3))

Int64Index([6, 4, 0], dtype='int64')


In [29]:
df.loc[:, 6]

0    0.908361
1    0.580314
2    0.668276
3    0.537522
4    0.097796
Name: 6, dtype: float64

In [30]:
df

,0,1,2,3,4,5,6,7,8,9
0,0.251386,0.336235,0.022106,0.744963,0.567653,0.701348,0.908361,0.981125,0.313437,0.190798
1,0.860694,0.979427,0.217470,0.738929,0.145392,0.167757,0.580314,0.226132,0.558793,0.094810
2,0.736175,0.175313,0.329451,0.936886,0.180998,0.985477,0.668276,0.062178,0.043345,0.996149
3,0.542475,0.326555,0.921905,0.928111,0.931490,0.122627,0.537522,0.971575,0.622160,0.723990
4,0.651818,0.032971,0.540289,0.774259,0.539623,0.022569,0.097796,0.401801,0.547276,0.523516
